Loading Dataset From Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Necessary Imports

In [2]:
import torch
from torchvision import datasets
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

Setting Hyper-Parameters

In [3]:
train_batch_size = 64
test_batch_size = 32
num_classes = 2
learning_rate = 0.001
num_epochs = 2
input_path = "/content/drive/MyDrive/MS Class Tutorial/C_D_Dataset/"

Setting Up Device (GPU/CPU)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Setting the Parameters to Normalise the Image

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

Data Transformation Parameters

In [6]:
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

Reading Images

In [7]:
image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'test': 
    datasets.ImageFolder(input_path + 'test', data_transforms['test'])
}

Data Loaders

In [8]:
dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=train_batch_size,
                                shuffle=True),
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=test_batch_size,
                                shuffle=True)
}

In [9]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()

        self.layer1 = self.conv_module(3, 16).to(device)
        self.layer2 = self.conv_module(16, 32).to(device)
        self.layer3 = self.conv_module(32, 64).to(device)
        self.layer4 = self.conv_module(64, 128).to(device)
        self.layer5 = self.conv_module(128, 256).to(device)
        self.gap = self.global_avg_pool(256, num_classes).to(device)

    def forward(self, x):
        out = self.layer1(x).to(device)
        out = self.layer2(out).to(device)
        out = self.layer3(out).to(device)
        out = self.layer4(out).to(device)
        out = self.layer5(out).to(device)
        out = self.gap(out).to(device)
        out = out.view(-1, num_classes).to(device)

        return out

    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)))

Inititialising the Model

In [10]:
model = ConvNeuralNet(num_classes).to(device)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [11]:
def train_model(model, criterion, optimizer, num_epochs):
	res_str = ""
	for epoch in range(num_epochs):
		print('Epoch {}/{}'.format(epoch+1, num_epochs))
		print('-' * 10)

		for phase in ['train', 'test']:
			if phase == 'train':
				model.train()
			else:
				model.eval()

			running_loss = 0.0
			running_corrects = 0

			for inputs, labels in dataloaders[phase]:
				inputs = inputs.to(device)
				labels = labels.to(device)

				outputs = model(inputs)
				loss = criterion(outputs, labels)

				if phase == 'train':
					optimizer.zero_grad()
					loss.backward()
					optimizer.step()

				_, preds = torch.max(outputs, 1)
				running_loss += loss.item() * inputs.size(0)
				running_corrects += torch.sum(preds == labels.data)

			epoch_loss = running_loss / len(image_datasets[phase])
			epoch_acc = running_corrects.double() / len(image_datasets[phase])

			print('{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
			res_str = res_str +str(epoch) +"\n" + '{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss, epoch_acc) + "\n"
	return model, res_str

In [12]:
model_trained, out_str = train_model(model, criterion, optimizer, num_epochs)
f = open("accuracy.txt", "w")
f.write(out_str)
f.close()

#SAVE THE MODEL
torch.save(model_trained.state_dict(), 'model_weights.h5')

Epoch 1/2
----------
train loss: 0.6732, acc: 0.5705
test loss: 0.7282, acc: 0.5030
Epoch 2/2
----------
train loss: 0.6590, acc: 0.5990
test loss: 0.6423, acc: 0.6350
